In [29]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

In [30]:
names = ['S' + str(format(i, '02')) for i in range(6)] # + 1 because range starts at 0
names.pop(0) # Pop the first element of the list to match the names of the files
print(names)

# Create a list of the variables and its columns
variables = ['EDA', 'HR', 'IBI','TEMP']
final_columns = {
    #'ACC': ['x', 'y', 'z'],
    'EDA': ['EDA'],
    'HR': ['HR',],
    'IBI': ['IBI', 'IBI_ms'],
    'TEMP': ['TEMP'],
}
print(variables)

['S01', 'S02', 'S03', 'S04', 'S05']
['EDA', 'HR', 'IBI', 'TEMP']


In [31]:
df_HR = pd.read_csv(f'https://raw.githubusercontent.com/italha-d/Stress-Predict-Dataset/main/Raw_data/S02/HR.csv',header=None)
pd.set_option('display.max_rows', 10)
start_time = df_HR.iloc[0,0]
df2 = df_HR.T
init = df2.pop(0)
hz = df2.pop(1)
df_HR = df2.T
time = [int(start_time+(i)) for i in range(len(df_HR))]
df_HR['time'] = time
#df_HR.set_index('time', inplace=True)
df_HR = df_HR.rename(columns={0: 'HR'}, inplace=False)
df_HR["label"] = " "
df_HR

,HR,time,label
2,118.00,1644227584,
3,113.50,1644227585,
4,93.00,1644227586,
5,93.25,1644227587,
6,86.40,1644227588,
...,...,...,...
3552,61.68,1644231134,
3553,62.03,1644231135,
3554,62.48,1644231136,
3555,62.88,1644231137,


In [32]:
# Import timelogs for visualization
# This DataFrame contains the time logs for each subject responses
timelogs = pd.read_csv('Time_logs.csv')
timelogs = timelogs.drop(0, axis=0) # Drop the first row
timelogs = timelogs.drop(['Session ID', 'Unnamed: 24'], axis=1)
timelogs

,S. ID.,Start Time,End Time,Consent,Unnamed: 5,Baseline/Questionniare,Unnamed: 7,Stroop Test,Unnamed: 9,Relax,...,Unnamed: 15,Hyperventilation,Unnamed: 17,Relax.2,Unnamed: 19,Questionniare,Unnamed: 21,Relax/Baseline,Unnamed: 23,Date
1,S01,9:22,10:22,9:22,9:27,9:39,9:43,9:29,9:34,9:34,...,9:59,10:00,10:02,10:02,10:07,10:08,10:12,10:12,10:22,7-Feb-2022
2,S02,9:48,10:51,9:48,9:50,9:52,10:02,10:03,10:08,10:08,...,10:29,10:29,10:31,10:31,10:36,10:36,10:41,10:41,10:51,7-Feb-2022
3,S03,10:50,11:50,10:50,10:53,10:55,11:02,11:04,11:09,11:09,...,11:28,11:28,11:30,11:30,11:35,11:35,11:40,11:40,11:50,7-Feb-2022
4,S04,11:25,12:28,11:25,11:30,11:30,11:40,11:40,11:45,11:45,...,12:05,12:05,12:07,12:07,12:12,12:12,12:18,12:18,12:28,7-Feb-2022
5,S05,9:05,10:06,9:05,9:10,9:12,9:22,9:23,9:29,9:29,...,9:49,9:49,9:51,9:51,9:56,9:56,10:01,10:01,10:06,14-Feb-2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,S31,3:05,4:04,3:05,3:10,3:10,3:20,3:21,3:26,3:27,...,3:47,3:47,3:49,3:49,3:54,3:54,3:57,3:57,4:04,7-Mar-2022
32,S32,3:34,4:30,3:34,3:37,3:37,3:47,3:47,3:53,3:53,...,4:13,4:13,4:15,4:15,4:20,4:20,4:24,4:24,4:30,7-Mar-2022
33,S33,12:11,1:08,12:11,12:14,12:15,12:25,12:24,12:30,12:30,...,12:50,12:51,12:53,12:53,12:58,12:58,1:05,1:05,1:08,9-Mar-2022
34,S34,2:30,3:35,2:30,2:35,2:38,2:50,2:51,2:57,2:57,...,3:18,3:18,3:20,3:20,3:25,3:25,3:30,3:30,3:35,9-Mar-2022


In [33]:
# Convert the time logs to unix timestamps
# This is done to be able to compare the time logs with the data
for column in timelogs.columns:
    unix_timestamps = []
    if column != 'Date' and column != 'S. ID.':
        timelogs[column] = timelogs.Date + ' ' + timelogs[column]
        times = timelogs[column].values.tolist()
        for date in times:
            dt_obj = datetime.strptime(date, '%d-%b-%Y %H:%M')
            unix_timestamps.append(int(dt_obj.timestamp()))
        timelogs[column] = unix_timestamps
timelogs = timelogs.drop(['Date', 'Start Time', 'End Time','Consent','Unnamed: 5'], axis=1)
timelogs

,S. ID.,Baseline/Questionniare,Unnamed: 7,Stroop Test,Unnamed: 9,Relax,Unnamed: 11,Interview,Unnamed: 13,Relax.1,Unnamed: 15,Hyperventilation,Unnamed: 17,Relax.2,Unnamed: 19,Questionniare,Unnamed: 21,Relax/Baseline,Unnamed: 23
1,S01,1644226740,1644226980,1644226140,1644226440,1644226440,1644226740,1644227040,1644227640,1644227640,1644227940,1644228000,1644228120,1644228120,1644228420,1644228480,1644228720,1644228720,1644229320
2,S02,1644227520,1644228120,1644228180,1644228480,1644228480,1644228780,1644228780,1644229440,1644229440,1644229740,1644229740,1644229860,1644229860,1644230160,1644230160,1644230460,1644230460,1644231060
3,S03,1644231300,1644231720,1644231840,1644232140,1644232140,1644232440,1644232440,1644232980,1644232980,1644233280,1644233280,1644233400,1644233400,1644233700,1644233700,1644234000,1644234000,1644234600
4,S04,1644233400,1644234000,1644234000,1644234300,1644234300,1644234600,1644234600,1644235200,1644235200,1644235500,1644235500,1644235620,1644235620,1644235920,1644235920,1644236280,1644236280,1644236880
5,S05,1644829920,1644830520,1644830580,1644830940,1644830940,1644831240,1644831240,1644831840,1644831840,1644832140,1644832140,1644832260,1644832260,1644832560,1644832560,1644832860,1644832860,1644833160
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31,S31,1646622600,1646623200,1646623260,1646623560,1646623620,1646623920,1646623920,1646624520,1646624520,1646624820,1646624820,1646624940,1646624940,1646625240,1646625240,1646625420,1646625420,1646625840
32,S32,1646624220,1646624820,1646624820,1646625180,1646625180,1646625480,1646625480,1646626080,1646626080,1646626380,1646626380,1646626500,1646626500,1646626800,1646626800,1646627040,1646627040,1646627400
33,S33,1646828100,1646828700,1646828640,1646829000,1646829000,1646829300,1646829300,1646829900,1646829900,1646830200,1646830260,1646830380,1646830380,1646830680,1646830680,1646787900,1646787900,1646788080
34,S34,1646793480,1646794200,1646794260,1646794620,1646794620,1646794980,1646794980,1646795580,1646795580,1646795880,1646795880,1646796000,1646796000,1646796300,1646796300,1646796600,1646796600,1646796900


In [34]:
# Get two values from df1
flag = 0
for index in range(timelogs.shape[1]):
    next_index = index + 1
    if index > 1 and index <= 17:
        val1 = timelogs.iloc[1, index]
        val2 = timelogs.iloc[1, next_index]
        print(val1)
        print(val2)
        print(flag)
        if flag == 0:
            df_HR.loc[(df_HR['time'] >= val1) & (df_HR['time'] <= val2), 'label'] = 'relaxed'
            flag = 1
        elif flag == 1:
            df_HR.loc[(df_HR['time'] >= val1) & (df_HR['time'] <= val2), 'label'] = 'stressed'
            flag = 0
# for col_index in range(timelogs.shape[1]):
#     next_col = col_index + 1
#     if column != 'S. ID.':
#         val1 = timelogs.iloc[1, column]
#         val2 = timelogs.iloc[1, next_col]
#         print(val1)
#         print(val2)
#val1 = timelogs.loc[1, 'Baseline/Questionniare']
#val2 = timelogs.loc[1, 'Unnamed: 7']
#val3 = timelogs.loc[1, 'Stroop Test']
#val4 = timelogs.loc[1, 'Unnamed: 9']

# Filter df2 to include only rows where 'value' is between val1 and val2 and change the label
#df_HR.loc[(df_HR['time'] >= val3) & (df_HR['time'] <= val4), 'label'] = 'stressed'
#range = df_HR[(df_HR['time'] >= val3) & (df_HR['time'] <= val4)]
pd.set_option('display.max_rows', None)
df_HR

1644228120
1644228180
0
1644228180
1644228480
1
1644228480
1644228480
0
1644228480
1644228780
1
1644228780
1644228780
0
1644228780
1644229440
1
1644229440
1644229440
0
1644229440
1644229740
1
1644229740
1644229740
0
1644229740
1644229860
1
1644229860
1644229860
0
1644229860
1644230160
1
1644230160
1644230160
0
1644230160
1644230460
1
1644230460
1644230460
0
1644230460
1644231060
1


,HR,time,label
2,118.00,1644227584,
3,113.50,1644227585,
4,93.00,1644227586,
5,93.25,1644227587,
6,86.40,1644227588,
7,81.83,1644227589,
8,79.71,1644227590,
9,78.12,1644227591,
10,76.67,1644227592,
11,75.60,1644227593,
